In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
from scipy.spatial import distance
from scipy.stats import pearsonr
import math

import os
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC, SVR
from sklearn_rvm import EMRVC
from sklearn_rvm import EMRVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import lightgbm as lgb

In [2]:
# ## 在線使用設置##############
# import plotly as py
# import plotly.graph_objs as go
# from plotly.offline import iplot, init_notebook_mode
# import plotly.express as px
# import cufflinks as cf
# from plotly.subplots import make_subplots
# cf.go_offline()
# cf.set_config_file(offline=False, world_readable=True)

# import matplotlib.pyplot as plt 


In [3]:
# linear interpolation(線性插值)
from scipy.interpolate import interp1d
def interpolate(x, kind='linear'):
    not_nan = np.logical_not(np.isnan(x))
    indices = np.arange(len(x))
#     interp = interp1d(indices[not_nan], x[not_nan], kind=kind)
    interp = interp1d(indices[not_nan], x[not_nan], kind=kind,fill_value="extrapolate")
    return interp(indices) 

In [4]:
# 評估績效
def MAPE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def nMAE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(y_true - y_pred))/y_true.mean() * 100

def RMSE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sqrt(((y_pred - y_true) ** 2).mean())

In [5]:
#日期轉換
def transform_day_of_year(day_of_year):
    if(day_of_year > 365):
        return (day_of_year-356)/(538-356)
    elif(day_of_year < 173):
        return (day_of_year+365-356)/(538-356)
    else:
        return 1-((day_of_year-173)/(355-172))   

In [6]:
#正規化
def data_normalize(raw, need_normalize):
    merge = raw.copy()
    #使用最大最小值進行標準化
    for i in range(len(need_normalize)):
        column = need_normalize[i]
        molecular = merge[column]-merge[column].min()
        denominator = merge[column].max()-merge[column].min()
        merge[column] = (molecular/denominator)
    return merge

In [7]:
#基本資料分割，依據天數
def split_data_day(merge, rows, few):
    row = rows.copy()
    data_merge = merge.copy()
    #print(data_merge)
#    取提前一天的資料
    day = rows['Date']-timedelta(days=few)
    datas = data_merge[data_merge['Date'].isin(day)]
    datas = datas[datas['Hour'].isin([row['Hour']])]
    if (len(datas)== 0):
        value = np.nan
        return value
    else:
        value = datas['power'].values[0]
        return value

def split_data_persistence(merge, rows,pattern,isHoliday=False,time=0):
    row = rows.copy()
    data_merge = merge.copy()
#     預測日為禮拜六時取前一個禮拜六，為國定假日時取過去最近一個禮拜日的資料，其餘使用提前一天的資料
    if isHoliday:
        if time ==0:
            if row['Weekday'][0] == 1 or row['Weekday'][0] == 6:
                day = rows['Date']-timedelta(days=7)
                datas = data_merge[data_merge['Date'].isin(day)]
                datas = datas[datas['Hour'].isin([row['Hour']])]
    #         elif row['Weekday'][0]==7:
    #             day = rows['Date']-timedelta(days=1)
    #             datas = data_merge[data_merge['Date'].isin(day)]
    #             datas = datas[datas['Hour'].isin([row['Hour']])]
            elif row['isholiday'][0]==1 and row['Weekday'][0]!=6 and row['Weekday'][0]!=7:
                row_date = rows['Date'].values[0]
                datas = data_merge[data_merge['Date']<row_date]
                datas = datas[datas['Weekday'].eq(7)]
                datas = datas[datas['Hour'].isin([row['Hour']])].reset_index()
                datas = datas.sort_values(by='TIME_TO_INTERVAL')
                datas = datas.iloc[-1:]
            else:
                day = rows['Date']-timedelta(days=1)
                datas = data_merge[data_merge['Date'].isin(day)]
                datas = datas[datas['Hour'].isin([row['Hour']])]
#         前一小時 time = 1 後一小時time = -1
        else:
           
            if row['Weekday'][0] == 1 or row['Weekday'][0] == 6:
                day = rows['Date']-timedelta(days=7)
                datas = data_merge[data_merge['Date'].isin(day)]
                datas = datas[datas['Hour'].isin([row['Hour']])]
                index =  datas.index
                datas =  data_merge.iloc[index-time]
            elif row['isholiday'][0]==1 and row['Weekday'][0]!=6 and row['Weekday'][0]!=7:
                row_date = rows['Date'].values[0]
                datas = data_merge[data_merge['Date']<row_date]
                datas = datas[datas['Weekday'].eq(7)]
                datas = datas[datas['Hour'].isin([row['Hour']])].reset_index()
                datas = datas.sort_values(by='TIME_TO_INTERVAL')
                datas = datas.iloc[-1:]
                datas = data_merge[data_merge['TIME_TO_INTERVAL'].isin(datas['TIME_TO_INTERVAL'])]
                datas = datas[datas['Hour'].isin([row['Hour']])]
                index =  datas.index
                datas =  data_merge.iloc[index-time]
            else:
                day = rows['Date']-timedelta(days=1)
                datas = data_merge[data_merge['Date'].isin(day)]
                datas = datas[datas['Hour'].isin([row['Hour']])]
                index =  datas.index
                datas =  data_merge.iloc[index-time]
                
            
    else:
        if (pattern==0):
            #周一和週六取前一個禮拜的資料，其餘取提前一天的資料
            if row['Weekday'][0] == 1 or row['Weekday'][0] == 6:
                day = rows['Date']-timedelta(days=7)
                datas = data_merge[data_merge['Date'].isin(day)]
                datas = datas[datas['Hour'].isin([row['Hour']])]
            else:
                day = rows['Date']-timedelta(days=1)
                datas = data_merge[data_merge['Date'].isin(day)]
                datas = datas[datas['Hour'].isin([row['Hour']])]
        else:
            #周一、周六、周日取前一個禮拜的資料，其餘取提前一天的資料
            if row['Weekday'][0] == 1 or row['Weekday'][0] == 6 or row['Weekday'][0]==7:
                day = rows['Date']-timedelta(days=7)
                datas = data_merge[data_merge['Date'].isin(day)]
                datas = datas[datas['Hour'].isin([row['Hour']])]
            else:
                day = rows['Date']-timedelta(days=1)
                datas = data_merge[data_merge['Date'].isin(day)]
                datas = datas[datas['Hour'].isin([row['Hour']])]
    if (len(datas)== 0):
        value = np.nan
        return value
    else:
        value = datas['power'].values[0]
        return value

In [8]:
# 設置輸入特徵
def set_input(target_day, target_inps):

# target_day: 預測天的資料(D+1)
# target_inps: 預測天輸入的特徵值(D+1)

#print(few_day[few_inps[0]].values)
# 設置歷史資料輸入特徵
# 設置預測天資料輸入特徵
    if len(target_inps)>1:
        target_features = np.concatenate((
                            [target_day[target_inps[fea]].values for fea in range(len(target_inps))]
                        ))
    elif len(target_inps)==1:
        target_features = target_day[target_inps[0]].values
#     inputs = hourly_attribute
#     return inputs
    return target_features

# 設置輸出特徵
def set_output(target_day):
    output = target_day['power'].values
    return output

# 設置預測天資訊
def set_idx(target_day):
    idx = {
#             'TIME_TO_INTERVAL': target_day_time['TIME_TO_INTERVAL'].tolist()[2],
        'TIME_TO_INTERVAL': target_day['TIME_TO_INTERVAL'].tolist()[0],
        'power': target_day['power'].tolist()[0],
        'isholiday' : target_day['isholiday'].tolist()[0],
#         'dayOfYear_t': target_day['dayOfYear_t'].tolist()[0],
    }
    return idx

In [9]:
def model_build(train_x, train_y, train_idx, test_x, test_y, test_idx, model_name,scaler_x,scaler_y):
    #模型訓練
    if model_name == 'xgb':
        model = xgb.XGBRegressor(objective='reg:squarederror',
                        learning_rate=0.01, 
                        max_depth=1,
                        colsample_bytree=0.1,
                        reg_lambda=0.01,
                        seed=1,
                        subsample=0.1,
                        min_child_weight=1,
                        n_estimators=4000).fit(train_x, train_y)
    elif model_name == 'lgb':
        model = lgb.LGBMRegressor(
               boosting_type='gbdt',
                     verbose = 0,
                     learning_rate = 0.01,
                     num_leaves = 35,
                     feature_fraction=0.8,
                     bagging_fraction= 0.9,
                     bagging_freq= 8,
                     lambda_l1= 0.6,
                     lambda_l2= 0).fit(train_x, train_y)
    elif model_name == 'svr':
        model = SVR(C=1, kernel="rbf", gamma='auto').fit(train_x, train_y)
    elif model_name == 'rvm':
        model = EMRVR(kernel="rbf", gamma='auto')
        model.fit(train_x, train_y)
    elif model_name == 'persistence':
        test_x = scaler_x.inverse_transform(test_x)
        test_idx['pred'] = test_x
        test_idx['true'] = test_y
        return test_idx



#     other_params = {'learning_rate': 0.1, 'n_estimators': 500, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
#     'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}
#     model = xgb.XGBRegressor(**other_params).fit(train_x, train_y)

# 預測
    pred_y = model.predict(test_x)
    
# 反正規劃
    pred_y = pred_y.reshape(-1,1)
    pred_y = scaler_y.inverse_transform(pred_y)
    test_idx['pred'] = pred_y
    test_idx['true'] = test_y
    return test_idx

In [10]:
def performance(preds, columns):
    mape, rmse, mae =  [], [], []
    euclidean, pear =  [], []
    for i in preds:
        pred = preds[i].dropna()
        mape.append(round(MAPE(pred['true'], pred[f'pred']),2))
        rmse.append(round(RMSE(pred['true'], pred[f'pred']),2))
        mae.append(round(nMAE(pred['true'], pred[f'pred']),2))
#     #歐氏距離越大，兩個用戶相似度就越小
#     euclidean.append(round(distance.euclidean(pred['true'], pred[f'pred']),2))
#     pear.append(round(pearsonr(pred['true'], pred[f'pred'])[0],2))
    
    pred_result = pd.DataFrame({'feature': columns,
              'P(RMSE)': rmse, 'P(MAPE)': mape, 'P(MAE)': mae, 
#                         'euclidean':euclidean, 'pearsonr': pear,
#               'P(RMSE)': rmse, 'P(MAPE)': mape, 'P(MAE)': mae,
             })
    return pred_result


In [11]:
def remove_oulier(total_data,merge, test_split_date):
    total_data = total_data.copy()
    data = merge.copy()
    train_data = total_data[pd.to_datetime(total_data['Date'])<test_split_date]
    
    
    re_before = len(data)
    n=1.5
    Q3 = np.percentile(train_data['power'],75) 
    Q1 = np.percentile(train_data['power'],25)
    #IQR = Q3-Q1
    IQR = Q3 - Q1 
    
    #outlier step
    outlier_step = n * IQR
    dq3 = data[~(data['power'] < Q3 + outlier_step)]
    dq1 = data[~(data['power'] > Q1 - outlier_step)]
    
    outlier = pd.concat([dq3, dq1])
#     print(f'3 : {dq3}')
#     dq3['power'].iplot()
#     print(f'1 : {dq1}')
#     dq1['power'].iplot()

    #outlier = Q3 + n*IQR 
    data=data[data['power'] < Q3 + outlier_step]
    #outlier = Q1 - n*IQR 
    data=data[data['power'] > Q1 - outlier_step]


#     outlier_ = data['power'].describe().T['75%']*1.5
#     d0 = data[~(data['power'] >= 0)]
#     d1 = data[~(data['power'] < outlier_)]
#     print(len(d0), len(d1))
#     outlier = pd.concat([d0, d1])
#     data = data[data['power'] >= 0]
#     data = data[data['power'] < outlier_]
    
    re_after = len(data)

    print(f'移除前: {re_before}, 移除後: {re_after}, 共移除 {re_before-re_after} 筆') 
    print(f'IQR: {IQR}, Q3: {Q3}, Q1: {Q1}, outlier(1.5*IQR): {outlier_step}, Q3+outlier: { Q3 + outlier_step}, Q1-outlier: {Q1 - outlier_step}')
    
    
#     return data, outlier
    return data

    

In [12]:
#體感溫度計算
def apparent_temperature(Tem,RH,V):
    hpa = (float(RH)/100)*6.105*math.exp((17.27*float(Tem))/(237.7+float(Tem)))
    #print(hpa)
    AT = 1.07*float(Tem)+0.2*hpa-0.65*float(V)-2.7
    #print(AT)
    
    return AT

# 正式開始

In [13]:
#抓取日歷國定假日
import requests

page = 0
isHoliday = pd.DataFrame()
while True:
    url = f"https://data.ntpc.gov.tw/api/datasets/308DCD75-6434-45BC-A95F-584DA4FED251/json?page={page}&size=1000"
    res = requests.get(url)
    resJson = res.json()
    resJson2 = pd.DataFrame(resJson)
    isHoliday = pd.concat([isHoliday, resJson2])
    page+=1
    if resJson[-1]['date']>'2022':
        break

In [14]:
isHoliday['Date'] = pd.to_datetime(isHoliday['date'])
isHoliday['isholiday']=isHoliday.apply(lambda row: 1 if row['isholiday']=='是' else 0, axis=1)
#1為國定假日0則不是
isHoliday = isHoliday[['Date','isholiday']]
# isHoliday['Date'] = pd.to_datetime(isHoliday['Date'])
isHoliday = isHoliday[isHoliday['Date']>='2022']

# 特徵選擇

In [15]:
Hour = ['Hour_0.0', 'Hour_1.0', 'Hour_2.0', 'Hour_3.0', 'Hour_4.0', 'Hour_5.0',
       'Hour_6.0', 'Hour_7.0', 'Hour_8.0', 'Hour_9.0', 'Hour_10.0',
       'Hour_11.0', 'Hour_12.0', 'Hour_13.0', 'Hour_14.0', 'Hour_15.0',
       'Hour_16.0', 'Hour_17.0', 'Hour_18.0', 'Hour_19.0', 'Hour_20.0',
       'Hour_21.0', 'Hour_22.0', 'Hour_23.0']

Dayofweek = ['Dayofweek_0.0', 'Dayofweek_1.0', 'Dayofweek_2.0', 'Dayofweek_3.0',
       'Dayofweek_4.0', 'Dayofweek_5.0', 'Dayofweek_6.0']

Isholiday = ['Isholiday_0', 'Isholiday_1']

#few_input=['power']
#target_input=['feel_temp']
#target_input_2=['Hour', 'dayofweek', 'isholiday','feel_temp','pre_day_1','pre_day_7','pre_day_1_7_2','pre_day_1_7_3','pre_day_1_7_H']
target_input_2=[ 'Hour','dayofweek','isholiday','feel_temp','RH','pre_day_1','pre_day_7','pre_day_1_7_2','pre_day_1_7_3']
#target_input_2=['pre_day_7']
# target_input=['Hour', 'dayofweek', 'isholiday']
# neet_normalize = ['power', 'Hour', 'Month', 'Weekday', 'dayofweek', 'quarter', 'isholiday', 'temp', 'temp8', 'temp(cwb)']

# neet_normalize = ['power', 'Hour', 'Month', 'dayofweek', 'quarter', 'isholiday', 'temp(ncue)', 'temp(cwb)']
# neet_normalize = ['power', 'Hour', 'Month', 'dayofweek', 'quarter', 'isholiday', 'temp(cwb)']
neet_normalize = ['power', 'Hour', 'dayofweek','feel_temp','RH','pre_day_1','pre_day_7','pre_day_1_7_2','pre_day_1_7_3','pre_day_1_7_H']
#few_days_list = [1]

In [16]:
#設定輸出大小，以查看全部資料
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('max_colwidth',100)

In [ ]:
data_name = ['力行館.csv', '工學院.csv', '教學一錧.csv', '管理學院經世館.csv', 
             '第九宿舍.csv', '第十宿舍.csv', 
             '汙水處理廠.csv', '總變電站.csv', '育成中心.csv']
# data_name = ['力行館(temp).csv']
log_datas = {}
model_perf = {}
test_split_date = pd.to_datetime('2022-05-01')

for name in data_name:
    print(name[:-4])
    data_temp = pd.read_csv(f'./Dataset/20230118/merge/{name}')
    Observatory = pd.read_csv(f'./Dataset/solar_汙水廠(history).csv')
    data1 = data_temp.copy()
    data2 = Observatory.copy()
    #做欄位整理
    data1['TIME_TO_INTERVAL'] = pd.to_datetime(data1['TIME_TO_INTERVAL'])
    data2['TIME_TO_INTERVAL'] = pd.to_datetime(data2['TIME_TO_INTERVAL'])
    data1['Date'] = data1['TIME_TO_INTERVAL'].dt.date
    data1['Hour'] = data1['TIME_TO_INTERVAL'].dt.hour
    data2['Date'] = data2['TIME_TO_INTERVAL'].dt.date
    data2['Hour'] = data2['TIME_TO_INTERVAL'].dt.hour
    data1.loc[data1['Hour']==0,'Hour']=24
  
    #轉換型態
    data1['Date'] = data1['Date'].astype(str)
    data2['Date'] = data2['Date'].astype(str)
    #合併
    data_merge = data1.merge(data2, how="left",on=['Date','Hour'])
    total_data = data_merge.copy()
    data = data_merge.copy()
    data = data.drop("TIME_TO_INTERVAL_y", axis = 1)
    data.rename(columns = {'TIME_TO_INTERVAL_x':'TIME_TO_INTERVAL'}, inplace = True)
    data['Date'] = pd.to_datetime(data['TIME_TO_INTERVAL']).dt.date
#     #只抓取>20220608的資料
#     data = data[pd.to_datetime(data['Date']) >= pd.to_datetime('2022-06-08')]
# #     print( pd.to_datetime(data['TIME_TO_INTERVAL']).dt.date[0])
# #     data_start_date = pd.to_datetime(data['TIME_TO_INTERVAL']).dt.date[0]+datetime.timedelta(days=1)
#     data_start_date = pd.to_datetime(data.head(1)['TIME_TO_INTERVAL']).dt.date+datetime.timedelta(days=1)
#     print(type(data_start_date.values[0]))
#     data = data[pd.to_datetime(data['Date']) >= pd.to_datetime('2022-06-09')]
    
   


    data['TIME_TO_INTERVAL'] = pd.to_datetime(data['TIME_TO_INTERVAL'])
    data = data.sort_values(by='TIME_TO_INTERVAL')
    data['Date'] = data['TIME_TO_INTERVAL'].dt.date
    data['Hour'] = data['TIME_TO_INTERVAL'].dt.hour
    data['Year'] = data['TIME_TO_INTERVAL'].dt.year
    data['Month'] = data['TIME_TO_INTERVAL'].dt.month
    data['Day'] = data['TIME_TO_INTERVAL'].dt.day
    data['Weekday'] = data['TIME_TO_INTERVAL'].dt.weekday+1

    data['dayOfYear'] = pd.to_datetime(data['Date']).dt.dayofyear
    data['quarter'] = pd.to_datetime(data['Date']).dt.quarter
    data['dayofweek'] = pd.to_datetime(data['Date']).dt.dayofweek
    data['dayOfYear_t'] = data['dayOfYear'].apply(lambda DOY: transform_day_of_year(DOY))
    data['dayOfYear'] = data['dayOfYear']/365
    
    data['Date'] = pd.to_datetime(data['Date'])
    isHoliday['Date'] = pd.to_datetime(isHoliday['Date'])
    data = pd.merge(data,isHoliday,on='Date',how='left')
    data['isholiday'] = data.apply(lambda row: 1 if row['isholiday']==1 else 0, axis=1)
    data = data.sort_values(by='TIME_TO_INTERVAL')
    data = data[['TIME_TO_INTERVAL','Year','Month','Date','Weekday','Day','Hour','power','dayofweek','isholiday','ApparentTemperature(pred)[CWB]','Temperature(pred)[CWB]','RelativeHumidity(pred)[CWB]']]
    #將體感溫度、溫度、濕度改名
    data.rename(columns = {'ApparentTemperature(pred)[CWB]':'feel_temp'}, inplace = True)
    data.rename(columns = {'Temperature(pred)[CWB]':'Temperature'}, inplace = True)
    data.rename(columns = {'RelativeHumidity(pred)[CWB]':'RH'}, inplace = True)
    
    print(data.head(5))
    #異常值移除
    data = remove_oulier(total_data,data, test_split_date)
    data = data.dropna().reset_index(drop=True)
    print(f'len: {len(data)}, dropna len: {len(data.dropna())}')

    
 
    
    pre_power = ['pre_day_1','pre_day_7','pre_day_1_7_2','pre_day_1_7_3','pre_day_1_7_H','pre_similarity','next_similarity']
#     pre_power = ['pre_day_1_7_H']
    for j in range(len(pre_power)):
        pre_data_power=[]
        for i in range(len(data)):
            target_day = data.loc[i:i].reset_index(drop=True)
            if(j==0):
                few_day = split_data_day(data,target_day,1)
                pre_data_power.append(few_day)
            elif(j==1):
                few_day = split_data_day(data,target_day,7)
                pre_data_power.append(few_day)
            elif(j==2):
                few_day = split_data_persistence(data,target_day,0,isHoliday=False)
                pre_data_power.append(few_day)
            elif(j==3):
                few_day = split_data_persistence(data,target_day,1,isHoliday=False)
                pre_data_power.append(few_day)
            elif(j==4):
                few_day = split_data_persistence(data,target_day,0,isHoliday=True,time=0)
                pre_data_power.append(few_day)
            elif(j==5):
                few_day = split_data_persistence(data,target_day,0,isHoliday=True,time = 1)
                pre_data_power.append(few_day)
            elif(j==6):
                few_day = split_data_persistence(data,target_day,0,isHoliday=True,time = -1)
                pre_data_power.append(few_day)
          
        data[pre_power[j]] = pre_data_power
    data = data.dropna().reset_index(drop=True)
    print(f'len: {len(data)}, dropna len: {len(data.dropna())}')
    data.to_csv(f'./Dataset/20230118/final/{name}',index=False) 

力行館
     TIME_TO_INTERVAL  Year  Month       Date  Weekday  Day  Hour    power  \
0 2022-01-25 15:00:00  2022      1 2022-01-25        2   25    15  105.212   
1 2022-01-25 16:00:00  2022      1 2022-01-25        2   25    16  102.321   
2 2022-01-25 17:00:00  2022      1 2022-01-25        2   25    17   82.890   
3 2022-01-25 18:00:00  2022      1 2022-01-25        2   25    18   82.032   
4 2022-01-25 19:00:00  2022      1 2022-01-25        2   25    19   76.891   

   dayofweek  isholiday  feel_temp  Temperature    RH  
0          1          0       22.0         22.0  75.0  
1          1          0       22.0         22.0  75.0  
2          1          0       22.0         22.0  75.0  
3          1          0        NaN          NaN   NaN  
4          1          0        NaN          NaN   NaN  
移除前: 1132, 移除後: 1068, 共移除 64 筆
IQR: 21.310249999998845, Q3: 77.87249999999982, Q1: 56.56225000000097, outlier(1.5*IQR): 31.965374999998268, Q3+outlier: 109.83787499999809, Q1-outlier: 24.5968

len: 453, dropna len: 453
育成中心
     TIME_TO_INTERVAL  Year  Month       Date  Weekday  Day  Hour  power  \
0 2022-01-04 19:00:00  2022      1 2022-01-04        2    4    19  15.49   
1 2022-01-04 20:00:00  2022      1 2022-01-04        2    4    20  17.33   
2 2022-01-04 21:00:00  2022      1 2022-01-04        2    4    21  17.33   
3 2022-01-04 22:00:00  2022      1 2022-01-04        2    4    22  12.53   
4 2022-01-04 23:00:00  2022      1 2022-01-04        2    4    23  12.66   

   dayofweek  isholiday  feel_temp  Temperature  RH  
0          1          0        NaN          NaN NaN  
1          1          0        NaN          NaN NaN  
2          1          0        NaN          NaN NaN  
3          1          0        NaN          NaN NaN  
4          1          0        NaN          NaN NaN  
移除前: 1533, 移除後: 1523, 共移除 10 筆
IQR: 3.3700000000003474, Q3: 13.690000000000511, Q1: 10.320000000000164, outlier(1.5*IQR): 5.055000000000521, Q3+outlier: 18.74500000000103, Q1-outlier: 5.26

In [ ]:
data

In [ ]:
# data_name = ['力行館(temp).csv', '工學院(temp).csv', '教學一錧(temp).csv', '管理學院經世館(temp).csv', 
#               '第九宿舍(temp).csv', '第十宿舍(temp).csv', 
#               '汙水處理廠(temp).csv', '總變電站(temp).csv', '育成中心(temp).csv']
data_name = ['力行館(temp).csv']
test_split_date = pd.to_datetime('2022-05-01')
for target in target_input_2:
    log_datas = {}
    target_input = ['pre_day_1']
    target_input.append(target)
    for name in data_name:
        test_x, test_y, test_idx = [],[],[]
        train_x, train_y, train_idx = [],[],[]
        data = pd.read_csv(f'Dataset/merge/new/0727/{name}')
        for i in range(len(data)):
            target_day = data.loc[i:i].reset_index(drop=True)
            target_day['TIME_TO_INTERVAL'] = pd.to_datetime(target_day['TIME_TO_INTERVAL'])
            is_test_data = (target_day['TIME_TO_INTERVAL']>=test_split_date).values[0]
            inputs = set_input(target_day, target_input)
            output = set_output(target_day)
            idx = set_idx(target_day)
            if is_test_data:
                test_x.append(inputs)
                test_y.append(output)
                test_idx.append(idx)
            else:
                train_x.append(inputs)
                train_y.append(output)
                train_idx.append(idx)
    
    #fit_transform 对数据先拟合 fit，找到数据的整体指标，如均值、方差、最大值最小值等，
    #然后对数据集进行转换transform，从而实现数据的标准化、归一化操作。
        scaler_x = MinMaxScaler()
        scaler_x.fit(train_x)
        train_x = scaler_x.transform(train_x)
        test_x = scaler_x.transform(test_x)
        scaler_y = MinMaxScaler()
        scaler_y.fit(train_y)
        train_y = scaler_y.transform(train_y)

        train_x, train_y = np.array(train_x), np.array(train_y)
        test_x, test_y = np.array(test_x), np.array(test_y)
        train_idx, test_idx = pd.DataFrame(train_idx), pd.DataFrame(test_idx)   


        #pred = model_build(train_x, train_y, train_idx, test_x, test_y, test_idx, 'persistence',scaler_x,scaler_y)
        #pred = model_build(train_x, train_y, train_idx, test_x, test_y, test_idx, 'svr',scaler_x,scaler_y)
        pred = model_build(train_x, train_y, train_idx, test_x, test_y, test_idx, 'rvm',scaler_x,scaler_y)
        #pred = model_build(train_x, train_y, train_idx, test_x, test_y, test_idx, 'xgb',scaler_x,scaler_y)
        #pred = model_build(train_x, train_y, train_idx, test_x, test_y, test_idx, 'lgb',scaler_x,scaler_y)

        print(f'train: {len(train_idx)}, test: {len(test_idx)}')
        log_datas[name[:-4]] = pred
    result = performance(log_datas, data_name)
    print(result)


In [ ]:
train_x

In [ ]:
# dataframe->array
performance(log_datas, data_name)

# train_y = train_y * (target_max - target_min) + target_min
# train_idx['power(n)'] = train_y
# 	力行館(temp).csv	10.46	32.85	28.71

In [ ]:
test_data = test_idx
test_idx

In [ ]:
# import plotly.graph_objects as go
# import matplotlib.dates as md
# data_name = ['力行館(temp).csv', '工學院(temp).csv', '教學一錧(temp).csv', '管理學院經世館(temp).csv', 
#               '第九宿舍(temp).csv', '第十宿舍(temp).csv', 
#               '汙水處理廠(temp).csv', '總變電站(temp).csv', '育成中心(temp).csv']
# line_color = [
#     '#1f77b4',  # muted blue
#     '#ff7f0e',  # safety orange
#     '#2ca02c',  # cooked asparagus green
#     '#d62728',  # brick red
#     '#9467bd',  # muted purple
#     '#8c564b',  # chestnut brown
#     '#e377c2',  # raspberry yogurt pink
#     '#7f7f7f',  # middle gray
#     '#bcbd22',  # curry yellow-green
#     '#17becf'   # blue-teal
# ]
# i=-1
# for name in data_name:
#     i+=1
#     merge_data = pd.read_csv(f'Dataset/merge/new/0727/{name}')
#     data = merge_data.copy()
#     xtick = int(len(data['TIME_TO_INTERVAL'])/72)

#     fig_line = go.Figure()
#     fig_line.add_trace(go.Scatter(y = data['power'], x=data['TIME_TO_INTERVAL'],
#                         mode='lines',
#                         name='真實值',
#                         #line={'dash': 'dash'},
#                         line_color= line_color[i]))
#     # fig_line.add_trace(go.Scatter(y = data['power'], x=data['TIME_TO_INTERVAL'],
#     #                     mode='lines',
#     #                     name='預測值',
#     #                     line_color= '#3498db'))
#     fig_line.update_layout(
#         yaxis_title='用電量/kWh',
#         xaxis_title='Date',
#         title=name,
#         font=dict(
#             size=18,
#         ),
#     #     yaxis2=dict(anchor='x', overlaying='y', side='right')
#         height=450, 
#         width=1500,

#     )

#     fig_line.update_xaxes(nticks=xtick)


#     #     fig_line.write_html(f'{folder_path}/img/{methods}_{i}.html')

#     fig_line.show()

In [ ]:
# 新舊資料相加
# concate
for filename in os.listdir(f"./新資料"): 
    sales1 =  pd.read_csv(f'./舊資料/{filename}')
    sales2 =  pd.read_csv(f'./新資料/{filename}')
    merge_data = pd.concat([sales1,sales2], axis=0, ignore_index=True)
    print(merge_data.head(10))
    print(merge_data.tail(10))
    merge_data = merge_data.reset_index()
    merge_data.to_csv(f'./final/{filename}', index=False)